    We had a trained model that classifies images of 'with mask' and 'without mask'. We now use haarcascade frontal face xml file to detect faces from video captured by our webcam and apply model to detect whether the captured face is 'with mask' or 'without mask'.

In [1]:
import cv2
import numpy as np
from keras.models import load_model
from keras.preprocessing import image

In [2]:
# Dowmload Haarcascade file of frontal face to detect face area
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [3]:
# load cnn model 
model = load_model('mask_detect.h5')

In [ ]:
# for webcam
cap=cv2.VideoCapture(0)

while True:
    bool_,img=cap.read()# captures frame and returns boolean value and captured image
    if not bool_:
        continue
    # convert captured video to gray scale
    gray_img= cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # use cascade to mark face in video
    faces_detected = face_cascade.detectMultiScale(gray_img, 1.32, 5)


    # apply model to marked faces to predict wearing of mask or not by preprocessing the image  
    for (x,y,w,h) in faces_detected:
        roi_gray=gray_img[y:y+w,x:x+h]#cropping region of interest i.e. face area from  image
        roi_gray=cv2.resize(roi_gray,(100,100))
        img_pixels = image.img_to_array(roi_gray)
        img_pixels = np.expand_dims(img_pixels, axis = 0)
        img_pixels /= 255
        predictions = model.predict(img_pixels)

        #find max indexed array
        max_index = np.argmax(predictions[0])

        emotions = ('WithMask','WithoutMask')
        predicted_emotion = emotions[max_index]
        color = [0,255,0] if predicted_emotion == 'WithMask' else [0,0,255]
        
        # marking rectangle around face printing text 'With mask' or 'Without mask'
        cv2.rectangle(img,(x,y),(x+w,y+h),color,thickness=5)
        cv2.putText(img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    resized_img = cv2.resize(img, (700, 500))
    cv2.imshow('Mask or no mask detection',resized_img)



    if cv2.waitKey(1) == ord('q'):# wait until 'q' key is pressed 
        break

cap.release()
cv2.destroyAllWindows()


###### In this way we can detect whether faces from video captured are wearing masks or not